<a href="https://colab.research.google.com/github/hangnadi/sandbox-python/blob/regresi_linear/Regresi_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresi Linear
---

Problem Statement: Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

Dataset: https://www.kaggle.com/datasets/camnugent/california-housing-prices.

Task: Gunakan regresi untuk memprediksi nilai *median_house_value*

---

## A. Persiapan Environment
1. (Opsi 1) File disimpan pada google drive
- Menghubungkan google colab dengan google drive
- Memindahkan direktori kerja ke folder tempat dataset tersimpan
- Import python libraries yang digunakan
- Load data sebagai python dataframe

In [ ]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Move working directory to yours (hint: use terminal command)
%cd #{your working directory}
%pwd

In [ ]:
# Import python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

In [ ]:
# Load dataset into pandas dataframe format (hint: use data structure from panda)
df = pd.read_csv("{your file's name}")

2. (Opsi 2) File langsung diupload
- Mengupload file
- Load data sebagai python dataframe

In [ ]:
# Upload file directly
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load dataset into pandas dataframe format (hint: use data structure from panda)
df = pd.read_csv("{your file's name}")

3. Cek dataframe

In [ ]:
# Call the previously created dataframe
df

## B. Analisa dan Eksplorasi Data
1. Rangkuman umum dataset
2. Rangkuman statistik data
3. Visualisasi data

Pada bagian ini dilakukan visualisasi dan analisa mengenai keadaan data. Data-data yang dirasa mengganggu proses pengolahan selanjutnya akan dihapus. Berikut temuan-temuan yang diperoleh pada proses analisa ini.

In [ ]:
# Dataset summary
df.info()

Data yang digunakan memiliki 20640 baris dan 10 kolom. Salah satu kolom yaitu *median_house_value* merupakan kolom nilai yang dijadikan sebagai variabel yang diprediksi. Sementara itu, kolom-kolom lain akan dipilih dan digunakan sebagai fitur-fitur yang digunakan untuk proses prediksi

---

Dari pemaparan data diatas ditemui pula bahwa:


1.   Variabel *total_bedrooms* memiliki 207 data yang tidak memiliki nilai (*null*). Pengolahan lebih lanjut dari masalah ini akan dilakukan pada bagian selanjutnya
2.   Semua data dari variabel kecuali variabel *ocean_proximity* memiliki nilai dengan tipe data "float64". Untuk itu pada bagian selanjutnya nilai pada  variabel *ocean_proximity* ini akan dikuantisasi dan direpresentasikan dengan angka agar dapat digunakan sebagai variabel prediksi

---

In [ ]:
# Dataset statistic summary
df.describe()

Tampak dari hasil tersebut, data memiliki rentang nilai yang tidak sama. Proses normalisasi dapat diterapkan agar proses pelatihan yang akan dilakukan pada bagian selanjutnya dapat dilakukan dengan lebih efisien.

In [ ]:
# Data Visualization
df.hist(bins = 50 , figsize=(10 , 6))

## C. Data Preparation
1. Pembersihan Null value
2. Pembagian data menjadi data latih, validasi, dan uji

Data null yang terdapat pada variabel *total_bedrooms* jumlahnya relatif sedikit (sekitar 1%). Pada pekerjaan ini, baris yang mengandung nilai *null* akan dihilangkan.

---

In [ ]:
# Null cleaning
df_clean = df.dropna()

In [ ]:
# Check new cleaned dataframe
df_clean.info()

Data yang telah dibersihkan akan dibagi dengan proporsi 70% data latih dan 30% data untuk pengujian

In [ ]:
# Split data to train, validation, and test set
df_train, df_test = train_test_split(df_clean , test_size= 0.3 , random_state=15)

In [ ]:
# Print the result
print("Banyak data latih: ", df_train.shape[0], "; uji: ", df_test.shape[0])
print("Proporsi data latih: ", round(df_train.shape[0]/df_clean.shape[0]*100.0, 2), "%; uji: ", round(df_test.shape[0]/df_clean.shape[0]*100.0, 2), "%")

Selanjutnya, **data latih** ini yang akan dipilih, dianalisis, dan diproses sebagai variabel input untuk proses regresi

## D. Kasus Regresi *Single Variable*

---

Pada regresi *single variable*, output 𝒚 diperoleh dari hasil perkalian antara nilai bobot $w$ dan satu variabel masukan $x$ serta nilai bias $b$. Atau dapat pula dinyatakan dengan

$y = xw + b$

Nilai $y$ adalah nilai dari *median_house_value* sementara masukan $x$ akan ditentukan berdasar proses analisis. Nilai bobot $w$ akan ditentukan dengan proses pelatihan menggunakan data latih. Algoritma *gradient_descent* digunakan dalam proses pelatihan ini. Untuk itu, pada bagian ini, tahapan-tahapan yang dilakukan adalah:
1. Analisis dan pemilihan variabel masukan
2. Normalisasi data
3. Proses pelatihan dengan data latih
4. Proses pengujian dengan data uji

### 1. Analisis dan Pemilihan Variabel Masukan

Analisis dilakukan dengan memperhatikan korelasi antara kandidat variabel masukan dengan variabel *median_house_value*. Nilai korelasi ini cukup penting mengingat model prediksi yang digunakan adalah regresi linear. Semakin baik korelasi antara dua data, maka hasil yang diperoleh dengan metode regresi linear juga akan semakin baik.

In [ ]:
# Variable correlation evaluation (graph)
attributes = ["longitude", "latitude", "total_bedrooms", "population", "households", "median_house_value", "median_income", "total_rooms", "ocean_proximity", "housing_median_age"]
a = pd.plotting.scatter_matrix(df_train[attributes], alpha=0.2, figsize=(12, 8))

Dari visual diatas, pola tren yang ditunjukkan antara variabel *median_house_value* dan *median_income* lebih jelas dibanding dengan variabel lain. Ini mengindikasikan bahwa *median_house_value* memiliki korelasi yang baik terhadap *median_income*

---


In [ ]:
# Variable correlation evaluation (numeric)
corr = df_train.corr()
corr['median_house_value'].sort_values(ascending=True)

Dari hasil diatas hipotesis bahwa variabel *median_income*  memiliki korelasi yang paling baik dapat tervalidasi. Variabel ini yang akan digunakan sebagai variabel masukan untuk kasus regresi *single variable*

In [ ]:
# Copy variable with the best correlation
df_train_used_single = df_train[["median_income", "median_house_value"]].copy()

In [ ]:
# Visualize them
a = df_train_used_single.hist(bins = 50 , figsize=(10 , 6))

### 2. Normalisasi Data

Pada proses normalisasi data ini, nilai data masukan diolah sedemikian sehingga memiliki nilai rataan pada 0 dan standar deviasi pada 1. Dengan demikian didapat nilai dari rataan $\mu$ dan *standard deviation* $\sigma$ dari variabel masukan $x$ sedemikian sehingga $\frac{(x-\mu)}{\sigma}$ memiliki rataan pada 0 dan standar deviasi pada 1.

Sementara itu data output juga diperlakukan dengan perlakuan yang sama dengan nilai rataan $\mu_y$ dan *standard deviation* $\sigma_y$

In [ ]:
# Check the statistical summary of the isolated variable dataframe
df_train_used_single.describe()

In [ ]:
# Get the mean and standard deviation value
mu_x = [df_train_used_single.describe().median_income[1]]
print("mean_data input: ", mu_x)
sigma_x = [df_train_used_single.describe().median_income[2]]
print("std_data input: ", sigma_x)

mu_out  = df_train_used_single.describe().median_house_value[1]
print("mean_data output: ", mu_out)
sigma_out = df_train_used_single.describe().median_house_value[2]
print("std_data output: ", sigma_out)

In [ ]:
# Normalize the data
df_train_normalized_single = df_train_used_single.copy()
df_train_normalized_single["median_income"] = (df_train_used_single["median_income"]-mu_x[0])/sigma_x[0]
df_train_normalized_single["median_house_value"] = (df_train_used_single["median_house_value"]-mu_out)/sigma_out

In [ ]:
# Show the mean and standar deviation value of normalized data
df_train_normalized_single.describe()

### 3. Training Model

- **Rumus penting:** <br>

a. *Loss Function*

Digunakan *mean squared error* yang dapat dinyatakan dengan persamaan

$F = \frac{1}{n} Σ_n (y_{pred}-y_{actual})^2 $

In [ ]:
def mse_loss(y_pred, y_actual):
  return ((y_pred-y_actual)**2).mean()

b. *Forward Propagation*

$y = \bar{x}\bar{w} + b$

Untuk kasus *single variable* diperoleh

$y = x_1w_1 +  b$

Sedangkan untuk kasus *multi varieble* diperoleh

$y = x_1w_1 + x_2w_2 +...+ x_nw_n + b$

Definisi fungsi berikut dapat digunakan untuk kasus *single variable* maupun *multi variable*

In [ ]:
def forward_pass(x_data, w_data, b_data):
  return np.dot(x_data, w_data) + b_data

c. *Backward Propagation*

Sesuai dengan algoritma *gradient descent*, untuk memperbaharui nilai bobot $w_i$, dimiliki persamaan

$w_i[k+1] = w_i[k]-a\frac{\partial{F}}{\partial{w_i}}$

diperoleh

$w_i[k+1] = w_i[k]-2a\times (y_{pred}-y_{actual})\times x_i$

dengan $a$ adalah nilai *learning rate*. Diperoleh pula

$b[k+1] = b[k]-2a\times (y_{pred}-y_{actual})$

In [ ]:
def backward_pass(w_data, x_data, b_data, a, y_pred, y_actual):
  multiplier = 2*a*(y_pred-y_actual)
  return w_data - np.dot(multiplier, x_data), b_data - multiplier.sum()

- Proses training
Algoritma *gradient descent* diterapkan dan diiterasi sebanyak 100 kali. *Learning rate* ditetapkan bernilai 0.000001. Nilai bobot dan awal ditetapkan seragam dengan nilai 0.0

In [ ]:
# Parameter and initialization
lr = 0.000001 #learning rate
w = [0.0] #weight
b = 0.0 #bias

epoch = 100

error_log = []

# Convert data into matrix
df_train_input = df_train_normalized_single.copy()
X = df_train_input.drop(["median_house_value"], axis=1).to_numpy()
y_actual = df_train_normalized_single.median_house_value.to_numpy()

# Start gradient descent
for i in range(epoch):
  y_pred = forward_pass(X, w, b)
  error = mse_loss(y_pred, y_actual)
  w, b = backward_pass(w, X, b, lr, y_pred, y_actual)

  error_log.append(error)

# Get the latest coefficient value
w_result = w
b_result = b

# Plotting
plt.plot(error_log)
plt.xlabel("Iteration")
plt.ylabel("Loss value")

In [ ]:
# Print last error val
print("Latest error: ", error_log[-1])
print("Weight value: ", w_result)
print("Bias value: ", b_result)

### 4. Pengujian dan Evaluasi

Karena terdapat tahapan normalisasi pada data latih. Maka pada proses pengujian, perlu dilakukan beberapa penyesuaian yaitu:

1. Data input pengujian harus dinormalisasi dengan nilai *mean* dan *standard deviation* yang diperoleh pada data latih
2. Data output hasil prediksi harus dikali dan ditambah secara berturut-turut dengan nilai $\sigma_y$ dan $\mu_y$ yang diperoleh saat pelatihan

- Persiapan variabel input data uji

In [ ]:
# Get the latitude_longitude variable
df_test_temp = df_test.copy()
df_test_prepared = df_test_temp[["median_income", "median_house_value"]].copy()

# Normalize the input data
df_test_prepared["median_income"] = (df_test_prepared["median_income"]-mu_x[0])/sigma_x[0]

- Prediksi nilai output<br>

Dengan menggunakan bobot dan bias yang diperoleh saat pelatihan, dengan menggunakan regresi *single variable* diperoleh hasil prediksi dari data uji

Data output hasil prediksi harus dikali dan ditambah secara berturut-turut dengan nilai $\sigma_y$ dan $\mu_y$ karena saat proses pelatihan, telah dilakukan proses normalisasi terhadap nilai $y_{actual}$

In [ ]:
# Get the test data
X_test = df_test_prepared.drop(["median_house_value"], axis=1).to_numpy()
y_actual_test = df_test_prepared.median_house_value.to_numpy() #Original data

# Get the output
y_regressor = forward_pass(X_test, w_result, b_result)

# Multiply the output with the standard deviation and then add it with the mean
y_pred_test = y_regressor*sigma_out + mu_out

- Hasil dari data uji
Berikut hasil plot yang membandingkan hasil prediksi dan nilai actual dari beberapa sampel data uji.

In [ ]:
# Plot the result of some sample data
plt.figure(figsize=(12, 3))
plt.plot(y_pred_test[0:20], 'ro', label="prediction", markersize=5)
plt.plot(y_actual_test[0:20], 'bo', label="actual", markersize=5)
plt.legend()

plt.ylabel("house_median_value")
plt.xlabel("data")

In [ ]:
# Loss value
test_error = mse_loss(y_regressor, (y_actual_test-mu_out)/sigma_out) #Use normalized actual data

print("Loss value in test data is:", test_error)

Dari hasil tersebut diperoleh *loss value* dari data uji ((0.55) tidak terlalu terpaut jauh dengan data latih (0.53). Dengan demikian telah diperoleh koefisien regresi yang telah cukup mewakili keseluruhan data (terutama data uji) walaupun hasil prediksi yang diperoleh belum secara signifikan sesuai dengan nilai aktual